Вы должны разработать скрипт на Python для парсинга таблиц с веб-сайтов.
Требования:
1. Скрипт должен принимать входной URL веб-страницы с таблицей.
2. Скрипт может использовать библиотеку requests для получения HTML-кода страницы.
3. Скрипт может использовать библиотеку BeautifulSoup для парсинга HTML и извлечения таблицы.
Допускается использование selenium.
4. Данные из таблицы должны быть извлечены и сохранены в формате Excel с использованием библиотек openpyxl или pandas.
5. Второй сайт является необязательным заданием, но будет плюсом. Если необходимо загрузить что-то с сайта, то это не обязательно оформлять в формате кода, но описать, как это можно сделать.
Допускается использование библиотек на ваше усмотрение. 

Первый сайт:
https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/de-lu/daily/?view=table

https://data.nordpoolgroup.com/auction/day-ahead/prices?deliveryDate=2024-02-16&deliveryAreas=AT&currency=EUR&aggregation=Hourly


In [1]:
import re
import requests
import json
import datetime
import pandas as pd


def get_a_data(url):
    date_areas_currency = re.search(r'.*deliveryDate=(.*)&deliveryAreas=(.*)&currency=(.*)&', url)
    delivery_date = date_areas_currency.group(1)
    delivery_area = date_areas_currency.group(2)
    currency = date_areas_currency.group(3)
    url_api = f'https://dataportal-api.nordpoolgroup.com/api/DayAheadPrices?date={delivery_date}&market=DayAhead&deliveryArea={delivery_area}&currency={currency}'
    data = requests.get(url_api).json()
    df = pd.json_normalize(data["multiAreaEntries"])
    return df

def format_time(date_str):
    datetime_obj = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    return datetime_obj.strftime("%H:%M")


base_url = 'https://data.nordpoolgroup.com/auction/day-ahead/prices?deliveryDate=2024-02-16&deliveryAreas=AT&currency=EUR&aggregation=Hourly'
df = get_a_data(base_url)

df['deliveryStart'] = df['deliveryStart'].apply(format_time)
df['deliveryEnd'] = df['deliveryEnd'].apply(format_time)
df['Delivery period (EET)'] = df['deliveryStart'].astype(str) + ' - ' + df['deliveryEnd'].astype(str)

df = df.rename(columns = {'deliveryStart': 'Delivery Start', 'deliveryEnd': 'Delivery End', 'entryPerArea.AT': 'AT (EUR)'})
df = df[['Delivery period (EET)', 'AT (EUR)', 'Delivery Start', 'Delivery End']]
print(df)

df.to_excel('output_1.xlsx', index=False, engine='openpyxl')

   Delivery period (EET)  AT (EUR) Delivery Start Delivery End
0          23:00 - 00:00     51.76          23:00        00:00
1          00:00 - 01:00     51.08          00:00        01:00
2          01:00 - 02:00     49.92          01:00        02:00
3          02:00 - 03:00     48.59          02:00        03:00
4          03:00 - 04:00     50.18          03:00        04:00
5          04:00 - 05:00     52.53          04:00        05:00
6          05:00 - 06:00     70.10          05:00        06:00
7          06:00 - 07:00     74.63          06:00        07:00
8          07:00 - 08:00     70.50          07:00        08:00
9          08:00 - 09:00     64.84          08:00        09:00
10         09:00 - 10:00     54.37          09:00        10:00
11         10:00 - 11:00     53.00          10:00        11:00
12         11:00 - 12:00     52.33          11:00        12:00
13         12:00 - 13:00     52.31          12:00        13:00
14         13:00 - 14:00     53.78          13:00      